In [50]:
import time
import pandas as pd

# Table 1 Data
staff = [
    {
        "staff_id": 1,
        "name": "Dr. Smith",
        "specialty": "Cardiology",
        "available_days": ["Monday", "Wednesday", "Friday"]
    },
    {
        "staff_id": 2,
        "name": "Dr. Johnson",
        "specialty": "Surgery",
        "available_days": ["Tuesday", "Thursday"]
    },
    {
        "staff_id": 3,
        "name": "Nurse Brown",
        "specialty": "Anesthetist",
        "available_days": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    },
    {
        "staff_id": 4,
        "name": "Nurse Miller",
        "specialty": "Pediatrics",
        "available_days": ["Monday", "Wednesday", "Friday"]
    }
]
# Room data
rooms = [
    {
        "room_id": 101,
        "room_type": "Surgical Room",
        "available_days": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
        "shift": "Morning"
    },
    {
        "room_id": 102,
        "room_type": "ICU",
        "available_days": ["Tuesday", "Thursday"],
        "shift": "Night"
    },
    {
        "room_id": 103,
        "room_type": "Pediatrics",
        "available_days": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
        "shift": "Afternoon"
    }
]
# patients information
patients = [
    {"patient_id": "P001", "name": "John Doe", "required_specialty": "Surgery"},
    {"patient_id": "P002", "name": "Helen Jones", "required_specialty": "Pediatrics"},
    {"patient_id": "P003", "name": "Emily Johnson", "required_specialty": "Cardiology"},
    {"patient_id": "P004", "name": "Tom Miller", "required_specialty": "Surgery"},
    {"patient_id": "P005", "name": "June Larson", "required_specialty": "Surgery"},
]

df_patients = pd.DataFrame(patients)
df_staff = pd.DataFrame(staff)
df_rooms = pd.DataFrame(rooms)
print("=" * 5, "PATIENTS DATA", "=" * 5)
print(df_patients)
print("=" * 5, "STAFF DATA", "=" * 5)
print(df_staff)
print("=" * 5, "ROOMS DATA", "=" * 5)
print(df_rooms)


===== PATIENTS DATA =====
  patient_id           name required_specialty
0       P001       John Doe            Surgery
1       P002    Helen Jones         Pediatrics
2       P003  Emily Johnson         Cardiology
3       P004     Tom Miller            Surgery
4       P005    June Larson            Surgery
===== STAFF DATA =====
   staff_id          name    specialty  \
0         1     Dr. Smith   Cardiology   
1         2   Dr. Johnson      Surgery   
2         3   Nurse Brown  Anesthetist   
3         4  Nurse Miller   Pediatrics   

                                   available_days  
0                     [Monday, Wednesday, Friday]  
1                             [Tuesday, Thursday]  
2  [Monday, Tuesday, Wednesday, Thursday, Friday]  
3                     [Monday, Wednesday, Friday]  
===== ROOMS DATA =====
   room_id      room_type                                  available_days  \
0      101  Surgical Room  [Monday, Tuesday, Wednesday, Thursday, Friday]   
1      102           

In [51]:
# identify the room supported specialites
room_type_to_specialties = {
    "Surgical Room": ["Surgery", "Cardiology"],
    "Pediatrics": ["Pediatrics"],
    "ICU": ["Cardiology", "Surgery"]
}
for room in rooms:
    room_type = room["room_type"]

    # Assign supported specialties based on room type
    room["supported_specialties"] = room_type_to_specialties.get(room_type, [])

print(pd.DataFrame(rooms))


   room_id      room_type                                  available_days  \
0      101  Surgical Room  [Monday, Tuesday, Wednesday, Thursday, Friday]   
1      102            ICU                             [Tuesday, Thursday]   
2      103     Pediatrics  [Monday, Tuesday, Wednesday, Thursday, Friday]   

       shift  supported_specialties  
0    Morning  [Surgery, Cardiology]  
1      Night  [Cardiology, Surgery]  
2  Afternoon           [Pediatrics]  


In [52]:

# Algorithm 1：Greedy First-Fit

def greedy_schedule(patients, staff, rooms):
    schedule = []
    unscheduled = []

    # Track occupied rooms: (room_id, day)
    room_occupancy = set()
    staff_occupancy = set()

    for patient in patients:
        assigned = False

        for s in staff:
            if s["specialty"] != patient["required_specialty"]:
                continue
            for day in s["available_days"]:

                for r in rooms:
                    if patient["required_specialty"] not in r["supported_specialties"]:
                        continue

                    if day not in r["available_days"]:
                        continue

                    if (r["room_id"], day, r["shift"]) in room_occupancy:
                        continue

                    if (s["name"], day, r["shift"]) in staff_occupancy:
                        continue


                    # Assign
                    schedule.append({
                        "patient_id": patient["patient_id"],
                        "patient_name": patient["name"],
                        "staff_name": s["name"],
                        "room_id": r["room_id"],
                        "day": day,
                        "shift": r["shift"]
                    })

                    room_occupancy.add((r["room_id"], day, r["shift"]))
                    staff_occupancy.add((s["name"], day, r["shift"]))

                    assigned = True
                    break

                if assigned:
                    break
            if assigned:
                break

        if not assigned:
            unscheduled.append(patient["patient_id"])

    return schedule, unscheduled

start_time = time.perf_counter()

schedule, unscheduled = greedy_schedule(patients, staff, rooms)

end_time = time.perf_counter()
execution_time = end_time - start_time
# ===============================
# Print schedule as table
# ===============================
df_schedule = pd.DataFrame(schedule)

print("\n=== Greedy Scheduling Result ===")
if not df_schedule.empty:
    print(df_schedule.to_string(index=False))
else:
    print("No patients were scheduled.")

print("\nUnscheduled patients:", unscheduled)
print(f"\nAlgorithm execution time: {execution_time:.6f} seconds")



=== Greedy Scheduling Result ===
patient_id  patient_name   staff_name  room_id      day     shift
      P001      John Doe  Dr. Johnson      101  Tuesday   Morning
      P002   Helen Jones Nurse Miller      103   Monday Afternoon
      P003 Emily Johnson    Dr. Smith      101   Monday   Morning
      P004    Tom Miller  Dr. Johnson      102  Tuesday     Night
      P005   June Larson  Dr. Johnson      101 Thursday   Morning

Unscheduled patients: []

Algorithm execution time: 0.000034 seconds


In [53]:
def backtracking_schedule(patients, staff, rooms):
    schedule = []
    room_occupancy = set()
    staff_occupancy = set()

    def backtrack(index):
        if index == len(patients):
            return True  # All patients scheduled successfully

        patient = patients[index]

        for s in staff:
            if s["specialty"] != patient["required_specialty"]:
                continue

            for day in s["available_days"]:
                for r in rooms:
                    # Use supported_specialties mapping
                    if patient["required_specialty"] not in r["supported_specialties"]:
                        continue

                    if day not in r["available_days"]:
                        continue

                    if (r["room_id"], day, r["shift"]) in room_occupancy:
                        continue

                    if (s["name"], day, r["shift"]) in staff_occupancy:
                        continue

                    # Tentative assignment
                    schedule.append({
                        "patient_id": patient["patient_id"],
                        "patient_name": patient["name"],
                        "staff_name": s["name"],
                        "room_id": r["room_id"],
                        "day": day,
                        "shift": r["shift"]
                    })
                    room_occupancy.add((r["room_id"], day, r["shift"]))
                    staff_occupancy.add((s["name"], day, r["shift"]))

                    # Recursive call
                    if backtrack(index + 1):
                        return True

                    # Backtrack
                    schedule.pop()
                    room_occupancy.remove((r["room_id"], day, r["shift"]))
                    staff_occupancy.remove((s["name"], day, r["shift"]))

        return False  # No feasible assignment

    success = backtrack(0)
    return success, schedule



start_time_backtracking = time.perf_counter()
success, optimal_schedule = backtracking_schedule(patients, staff, rooms)
end_time_backtracking = time.perf_counter()


execution_time_backtracking = end_time_backtracking - start_time_backtracking

print("Solution exists:", success)
print("Schedule:", pd.DataFrame(optimal_schedule))
print(f"\nAlgorithm execution time: {execution_time_backtracking:.6f} seconds")


Solution exists: True
Schedule:   patient_id   patient_name    staff_name  room_id       day      shift
0       P001       John Doe   Dr. Johnson      101   Tuesday    Morning
1       P002    Helen Jones  Nurse Miller      103    Monday  Afternoon
2       P003  Emily Johnson     Dr. Smith      101    Monday    Morning
3       P004     Tom Miller   Dr. Johnson      102   Tuesday      Night
4       P005    June Larson   Dr. Johnson      101  Thursday    Morning

Algorithm execution time: 0.000031 seconds
